In [1]:
import pickle
import torch
import torch.nn as nn
import random
from torch.utils.data import DataLoader
import time

In [2]:
class BertDataset(torch.utils.data.Dataset):
    def __init__(self, file):
        super(BertDataset, self).__init__()
        with open(file, 'rb') as f:
            self.training_data = pickle.load(f)
    
    def __len__(self):
        return len(self.training_data)
    
    def __getitem__(self,index):    
        return self.training_data[index]['sentence'],self.training_data[index]['sentiment']         
    
    # def getbatch(self):
    #     batch_data,batch_labels = [],[]                            ## First list for samples, the second for labels
    #     maxval = 0.0                                               ## For approximate batch data normalization
    #     for _ in range(self.batch_size):
    #         item = self._getitem()
    #         if np.max(item[0]) > maxval: 
    #             maxval = np.max(item[0])
    #         batch_data.append(item[0])
    #         batch_labels.append(item[1])
    #     batch_data = [item/maxval for item in batch_data]          ## Normalize batch data
    #     batch = [batch_data, batch_labels]
    #     return batch

In [3]:
dataset = BertDataset('bert_embedding_classes.pkl')
dataloader = DataLoader(dataset, batch_size = 8,shuffle = True , drop_last = True)

In [4]:
# for data in dataloader:
#     sente,senti = data
#     # print(data)
#     print(sente)
#     print(senti)
#     break
# print(len(dataloader))

In [28]:
class GRUnet(torch.nn.Module):
            """
            Source: https://blog.floydhub.com/gru-with-pytorch/
            with the only modification that the final output of forward() is now
            routed through LogSoftmax activation. 

            Class Path: DLStudio  ->  TextClassification  ->  GRUnet
            """
            def __init__(self, input_size, hidden_size, output_size, num_layers,  batch_size, drop_prob=0.2):
                super(GRUnet, self).__init__()
                self.hidden_size = hidden_size
                self.num_layers = num_layers
                self.gru = nn.GRU(input_size, hidden_size, num_layers,batch_first=True, dropout=drop_prob)
                self.fc = nn.Linear(hidden_size, output_size)
                self.leakyrelu = nn.LeakyReLU()
                self.softmax = nn.Softmax(dim=1)
                self.batch_size = batch_size
                
            def forward(self, x, h):
                out, h = self.gru(x, h)
                out = self.fc(self.leakyrelu(out[:,-1]))
                out = self.softmax(out)
                return out, h

            def init_hidden(self):
                weight = next(self.parameters()).data
                #batch_size   
                hidden = weight.new(  self.num_layers,   self.batch_size,         self.hidden_size   ).zero_()
                return hidden

In [29]:
class GRUnetBiDir(torch.nn.Module):
            """
            Source: https://blog.floydhub.com/gru-with-pytorch/
            with the only modification that the final output of forward() is now
            routed through LogSoftmax activation.

            Class Path: DLStudio  ->  TextClassification  ->  GRUnet
            """
            def __init__(self, input_size, hidden_size, output_size, num_layers,  batch_size, drop_prob=0.2):
                super(GRUnet, self).__init__()
                self.hidden_size = hidden_size
                self.num_layers = num_layers
                self.gru = nn.GRU(input_size, hidden_size, num_layers,batch_first=True, dropout=drop_prob, bidirectional = True)
                self.fc = nn.Linear(hidden_size, output_size)
                self.relu = nn.ReLU()
                self.softmax = nn.Softmax(dim=1)
                self.batch_size = batch_size

            def forward(self, x, h):
                out, h = self.gru(x, h)
                out = self.fc(self.relu(out[:,-1]))
                out = self.softmax(out)
                return out, h

            def init_hidden(self):
                weight = next(self.parameters()).data
                #                                     batch_size
                hidden = weight.new(  self.num_layers,    1,         self.hidden_size   ).zero_()
                return hidden

In [30]:
gru_network = GRUnet(input_size = 768, hidden_size = 512,output_size = 3, num_layers = 5, batch_size = 8)

In [49]:
class BiGRUnet(torch.nn.Module):
            """
            Source: https://blog.floydhub.com/gru-with-pytorch/
            with the only modification that the final output of forward() is now
            routed through LogSoftmax activation.

            Class Path: DLStudio  ->  TextClassification  ->  GRUnet
            """
            def __init__(self, input_size, hidden_size, output_size, num_layers,  batch_size, drop_prob=0.2):
                super(BiGRUnet, self).__init__()
                self.hidden_size = hidden_size
                self.num_layers = num_layers
                self.gru = nn.GRU(input_size, hidden_size, num_layers,batch_first=True, dropout=drop_prob, bidirectional=True)
                self.fc = nn.Linear(2*hidden_size, output_size)
                self.relu = nn.ReLU()
                self.softmax = nn.Softmax(dim=1)
                self.batch_size = batch_size

            def forward(self, x, h):
                out, h = self.gru(x, h)
                out = self.fc(self.relu(out[:,-1]))
                out = self.softmax(out)
                return out, h

            def init_hidden(self):
                weight = next(self.parameters()).data
                #                                     batch_size
                hidden = weight.new(  2*self.num_layers,    self.batch_size,         self.hidden_size   ).zero_()
                return hidden

In [50]:
run_code_for_training_GRU(gru_network, dataloader,epochs = 5, learning_rate = 1e-3, momentum = 0.5, display_train_loss=False)


torch.Size([8, 1, 81, 768])
torch.Size([1, 81, 768])
sentiment
tensor([[1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.]])
output
tensor([[0.3420, 0.3262, 0.3318],
        [0.3446, 0.3260, 0.3294],
        [0.3446, 0.3271, 0.3283],
        [0.3445, 0.3286, 0.3269],
        [0.3456, 0.3277, 0.3267],
        [0.3458, 0.3265, 0.3277],
        [0.3456, 0.3248, 0.3296],
        [0.3435, 0.3287, 0.3279]], grad_fn=<SoftmaxBackward0>)
torch.Size([8, 1, 81, 768])
torch.Size([1, 81, 768])
sentiment
tensor([[0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.]])
output
tensor([[0.3426, 0.3287, 0.3287],
        [0.3434, 0.3277, 0.3289],
        [0.3467, 0.3228, 0.3305],
        [0.3448, 0.3249, 0.3303],
        [0.3429, 0.3284, 0.3286],
        [0.3412, 0.3263, 0.3325],


NameError: name 'self' is not defined

In [ ]:
def run_code_for_testing_GRU(net, dataloader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net = net.to(device)
    correct = 0
    total = 0
    confusion_matrix = torch.zeros(3, 3)
    class_correct = [0] * 3
    class_total = [0] * 3
    with torch.no_grad():
        for i,data in enumerate(dataloader):
            ##  data is set to the images and the labels for one batch at a time:
            sentence,sentiment = data
            sentence = sentence.to(device)
            sentiment = sentiment.to(device)
            # if i % 1000 == 999:
            #     print("\n\n[i=%d:] Ground Truth:     " % (i+1) + ' '.join('%5s' % self.class_labels[labels[j]]
            #                                                    for j in range(self.batch_size)))
            outputs, hidden = net(sentence)
            predicted = outputs
            for sentiment,prediction in zip(sentiment,predicted):
                    # print(prediction)
                    confusion_matrix[torch.argmax(sentiment)][torch.argmax(prediction)] += 1
                    correct += 1 if torch.argmax(sentiment) == torch.argmax(prediction) else 0
            total += sentiment.size(0)
            if (i%100 == 99):
                print(i)